### Set

In [ ]:
pip install transformers

In [1]:
import pandas as pd
import numpy as np
import os
import gc
from tqdm import tqdm
import random
from collections import Counter
from IPython.display import display


import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizerFast, AutoModel
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score

In [ ]:
device = torch.device('cuda', 0) if torch.cuda.is_available() else 'cpu'
print(device)

# BERT Tokenizer: 使用 'bert-base-chinese' 版本
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')

# BERT Model: 使用 'bert-base-chinese' 版本
bert = AutoModel.from_pretrained('bert-base-chinese').to(device)

cpu


In [ ]:
# config
config = {"embedding_path": "/content/drive/MyDrive/embedding_tensor.pt",
          "seed": 1006,
          "learning_rate": 2e-5,
          "batch_size": 128, #128 #256
          "epochs": 300}

In [ ]:
def set_random_seed(seed, deterministic=False):
    random.seed(seed)
    np.random.seed(seed)

set_random_seed(config["seed"])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
raw_data1 = pd.read_csv("/content/drive/MyDrive/IM 專題 - ESG rating system/data/News - 11_train_data.csv")
raw_data2 = pd.read_csv("/content/drive/MyDrive/IM 專題 - ESG rating system/data/News - 12_train_data.csv")
raw_data001 = pd.read_csv("/content/drive/MyDrive/IM 專題 - ESG rating system/data/news - 001.csv")
raw_data002 = pd.read_csv("/content/drive/MyDrive/IM 專題 - ESG rating system/data/news - 002.csv")
raw_data003 = pd.read_csv("/content/drive/MyDrive/IM 專題 - ESG rating system/data/news - 003.csv")
raw_data004 = pd.read_csv("/content/drive/MyDrive/IM 專題 - ESG rating system/data/news - 004.csv")
raw_data006 = pd.read_csv("/content/drive/MyDrive/IM 專題 - ESG rating system/data/news - 006.csv")
raw_data007 = pd.read_csv("/content/drive/MyDrive/IM 專題 - ESG rating system/data/news - 007.csv")
raw_data009 = pd.read_csv("/content/drive/MyDrive/IM 專題 - ESG rating system/data/news - 009.csv")
raw_data010 = pd.read_csv("/content/drive/MyDrive/IM 專題 - ESG rating system/data/news - 010.csv")
raw_data011 = pd.read_csv("/content/drive/MyDrive/IM 專題 - ESG rating system/data/news - 011.csv")
raw_data012 = pd.read_csv("/content/drive/MyDrive/IM 專題 - ESG rating system/data/news - 012.csv")

merged_data = pd.concat([raw_data1, raw_data2, raw_data001, raw_data002, raw_data003, raw_data004, raw_data006, raw_data007, raw_data009, raw_data010, raw_data011, raw_data012], axis=0, ignore_index=True)
raw_data = merged_data
raw_data

,Index,Name,Content,Note,Company,Tag01,Tag02,Tag03,Tag04,URL,Unnamed: 10,Unnamed: 9,Keyword,xx
0,NaN,「下半年實在看不清楚！」聯詠股東會烏雲裡仍有好消息：OLED Q4產能仍不足,驅動IC大廠聯詠（3034）今（8）日股東會，會中拍板去年現金股利51.5元，依照最新收盤價...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,股東會後，聯詠副董事長王守仁坦言，受到通膨因素、升息循環、俄烏衝突、中國封控等不確定因素干擾...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,面板從去年第三季就出現調節，聯詠第二季財測更正式預估毛利率將首度跌破50%，讓驅動IC市場傳...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,聯詠在第一季度法說即釋出能見度不明朗訊息，今日維持此看法，王守仁更近一步點出第三季不明朗的擔...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,王守仁列舉近期將因應需求不明朗而調整稼動率的大廠，例如三星將在六月關閉LCD廠、轉做OLED...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18051,198,防地緣政治風險 台積電：政府應建立完整供應鏈,盧志遠並說，希望政策能多開放吸引國外人才移民台灣，像AI晶片大廠輝達員工中有四成印度人；目前...,NaN,NaN,多樣性與共榮 - 1,勞工關係 - 1,NaN,NaN,NaN,NaN,NaN,旺宏,NaN
18052,199,防斷鏈 台積電：新封裝關鍵材料、設備應產學合作加強創新研發,〔記者洪友芳／台北報導〕台積電系統整合前瞻研發副總經理余振華今指出，地緣政治使得生產製造走向...,NaN,NaN,供應鏈管理 - 1,NaN,NaN,NaN,NaN,NaN,NaN,旺宏,NaN
18053,199,防斷鏈 台積電：新封裝關鍵材料、設備應產學合作加強創新研發,SEMICON Taiwan 2023國際半導體展今天下午舉行「半導體研發大師論壇」，由國科...,NaN,NaN,多樣性與共榮 - 1,NaN,NaN,NaN,NaN,NaN,NaN,旺宏,NaN
18054,200,盧志遠：台灣教授太低薪 每年應加薪15％,〔記者洪友芳／台北報導〕SEMICON Taiwan 2023國際半導體展今天下午舉行「半導...,NaN,NaN,薪酬與福利 - 1,NaN,NaN,NaN,NaN,NaN,NaN,旺宏,NaN


In [ ]:
num_of_content = len(raw_data)
num_of_content

18056

In [ ]:
# 在 for loop 中根據條件刪除行
# 刪除沒有 content 的 row
for index, row in raw_data.iterrows():
  if type(raw_data["Content"][index]) != str:
    raw_data = raw_data.drop(index)
raw_data = raw_data.reset_index(drop=True)

num_of_content = len(raw_data)

# 將每個段落都加上標題
# 同時將 note 複製
now = "now"
note = "note"
for index, row in raw_data.iterrows():
  if type(raw_data["Name"][index]) == str:
    now = raw_data["Name"][index]
    note = raw_data["Note"][index]
  if type(raw_data["Name"][index]) != str:
    raw_data["Name"][index] = now
    raw_data["Note"][index] = note

# 刪掉 note="無關公司","無關組織","無關產業"的
for index, row in raw_data.iterrows():
  if raw_data["Note"][index] == "無關公司" or raw_data["Note"][index] == "無關組織" or raw_data["Note"][index] == "無關產業":
    raw_data = raw_data.drop(index)
raw_data = raw_data.reset_index(drop=True)

<ipython-input-9-85b1d82acda3>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data["Name"][index] = now
<ipython-input-9-85b1d82acda3>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data["Note"][index] = note


In [ ]:
num_of_content = len(raw_data)
num_of_content

17426

In [ ]:
# 合併成以文章為單位
# 用 raw_data1 測試

""" provided by chatGPT
import pandas as pd

# 示例 DataFrame
data = {'Name': ['A', 'B', 'A', 'C'],
        'Content': ['2k5', '2k6', '2k7', '2k8'],
        'Tag01': ['', 'TagB', 'TagC', 'TagD']}

df = pd.DataFrame(data)

# 定义合并规则的函数
def custom_merge(group):
    name = group['Name'].iloc[0]
    content = ''.join(group['Content'].tolist())
    tag01 = group['Tag01'].iloc[0] if group['Tag01'].iloc[0] != '' else group['Tag01'].iloc[1]
    return pd.Series({'Name': name, 'Content': content, 'Tag01': tag01})

# 根据 'Name' 列分组，然后对每个分组应用自定义合并规则
df_result = df.groupby('Name', as_index=False).apply(custom_merge)

print(df_result)
"""

# 定义合并规则的函数
def custom_merge(group):
    name = group['Name'].iloc[0]
    content = ''.join(group['Content'].tolist())
    tag01 = group['Tag01'].iloc[0] if group['Tag01'].iloc[0] != '' else group['Tag01'].iloc[1]
    return pd.Series({'Name': name, 'Content': content, 'Tag01': tag01})

# 根据 'Name' 列分组，然后对每个分组应用自定义合并规则
#raw_data_document = raw_data.groupby('Name', as_index=False).apply(custom_merge)

#print(raw_data_document)

In [ ]:
"""
num_of_document = len(raw_data_document)
num_of_document
"""

'\nnum_of_document = len(raw_data_document)\nnum_of_document\n'

In [ ]:
# 確認是否與 疫情/選舉 有關
# 關鍵字可以再調整

import re

def check_substrings(main_string, substrings):
    for substring in substrings:
        pattern = re.compile(re.escape(substring))
        if pattern.search(main_string):
            return True
    return False

def is_covid(main_string):
  substrings = ["疫情", "防疫"]
  return check_substrings(main_string, substrings)

def is_election(main_string):
  substrings = ["選舉", "政治獻金"]
  return check_substrings(main_string, substrings)

In [ ]:
# 刪掉 "疫情" "選舉" 相關的整篇新聞
"""
delete_lis = []
for index, row in raw_data.iterrows():
  Name = raw_data["Name"][index]
  for index_, row_ in raw_data.iterrows():
    if raw_data["Name"][index_] == raw_data["Name"][index]:
        if is_covid(raw_data["Content"][index]) or is_election(raw_data["Content"][index]):
          delete_lis.append(index)
          delete_lis.append(index_)

delete_lis = list(set(delete_lis))
for index in range(len(delete_lis)):
  raw_data = raw_data.drop(delete_lis[index])
raw_data = raw_data.reset_index(drop=True)
"""

'\ndelete_lis = []\nfor index, row in raw_data.iterrows():\n  Name = raw_data["Name"][index]\n  for index_, row_ in raw_data.iterrows():\n    if raw_data["Name"][index_] == raw_data["Name"][index]:\n        if is_covid(raw_data["Content"][index]) or is_election(raw_data["Content"][index]):\n          delete_lis.append(index)\n          delete_lis.append(index_)\n\ndelete_lis = list(set(delete_lis))\nfor index in range(len(delete_lis)):\n  raw_data = raw_data.drop(delete_lis[index])\nraw_data = raw_data.reset_index(drop=True)\n'

In [ ]:
""""""
num_of_content = len(raw_data)
num_of_content


17426

In [ ]:
# filter
# ESG relation classifier
# 加上標籤：該篇文章有關/無關
# 將一筆資料改成 "一篇" 文章


# 創建欄位
num_of_content = len(raw_data)
lis = [0] * num_of_content
raw_data["Related"] = lis

for index, row in raw_data.iterrows():
    if type(raw_data["Tag01"][index]) != str:
        raw_data["Related"][index] = 0
    else:
        raw_data["Related"][index] = 1  # related paragraph

"""
# 改以文章為單位
lis = [0] * num_of_document
raw_data_document["Related"] = lis

for index, row in raw_data_document.iterrows():
    if type(raw_data_document["Tag01"][index]) != str:
        raw_data_document["Related"][index] = 0
    else:
        raw_data_document["Related"][index] = 1  # related paragraph
"""


"""
for index, row in raw_data.iterrows():
  Name = raw_data["Name"][index]
  for index_, row_ in raw_data.iterrows():
    if raw_data["Name"][index_] == raw_data["Name"][index]:  # 如果同篇文章中有段落是有關的，就也算有關 # 如果還是不行就試把文章段落合併
        if raw_data["Related"][index] == 2 and raw_data["Related"][index_] == 0:
          raw_data["Related"][index_] = 1  # 文章有關但該段落無關

# 刪掉 relavent doc 中 irre
for index, row in raw_data.iterrows():
  if raw_data["Related"][index] == 1:
      raw_data = raw_data.drop(index)
raw_data = raw_data.reset_index(drop=True)
"""
"""
# 刪掉 Related = 0
for index, row in raw_data.iterrows():
  if raw_data["Related"][index] == 0:
    raw_data = raw_data.drop(index)

for index, row in raw_data.iterrows():
  if raw_data["Related"][index] == 1:
    raw_data["Related"][index] = 0
  else:
    raw_data["Related"][index] = 1
"""
"""
# 單就有無分分看
for index, row in raw_data.iterrows():
  Name = raw_data["Name"][index]
  for index_, row_ in raw_data.iterrows():
    if raw_data["Name"][index_] == raw_data["Name"][index]:  # 如果同篇文章中有段落是有關的，就也算有關 # 如果還是不行就試把文章段落合併
        if raw_data["Related"][index] == 1 and raw_data["Related"][index_] == 0:
          raw_data["Related"][index_] = 1  # 文章有關但該段落無關
"""

<ipython-input-16-5e265e622dd9>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data["Related"][index] = 0
<ipython-input-16-5e265e622dd9>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data["Related"][index] = 1  # related paragraph


'\n# 單就有無分分看\nfor index, row in raw_data.iterrows():\n  Name = raw_data["Name"][index]\n  for index_, row_ in raw_data.iterrows():\n    if raw_data["Name"][index_] == raw_data["Name"][index]:  # 如果同篇文章中有段落是有關的，就也算有關 # 如果還是不行就試把文章段落合併\n        if raw_data["Related"][index] == 1 and raw_data["Related"][index_] == 0:\n          raw_data["Related"][index_] = 1  # 文章有關但該段落無關\n'

In [ ]:
""""""
news_articles = raw_data["Name"]+raw_data["Content"]
label_data = raw_data.iloc[:,-1:]
label_data.to_csv('/content/drive/MyDrive/IM 專題 - ESG rating system/label_data_seg_all.csv', index=False)

#news_articles = raw_data_document["Name"]+raw_data_document["Content"]
#label_data = raw_data_document.iloc[:,-1:]
#label_data.to_csv('/content/drive/MyDrive/IM 專題 - ESG rating system/label_data_document_all.csv', index=False)

print(news_articles)
print(label_data)

0        「下半年實在看不清楚！」聯詠股東會烏雲裡仍有好消息：OLED Q4產能仍不足驅動IC大廠聯詠...
1        「下半年實在看不清楚！」聯詠股東會烏雲裡仍有好消息：OLED Q4產能仍不足股東會後，聯詠副...
2        「下半年實在看不清楚！」聯詠股東會烏雲裡仍有好消息：OLED Q4產能仍不足面板從去年第三季...
3        「下半年實在看不清楚！」聯詠股東會烏雲裡仍有好消息：OLED Q4產能仍不足聯詠在第一季度法...
4        「下半年實在看不清楚！」聯詠股東會烏雲裡仍有好消息：OLED Q4產能仍不足王守仁列舉近期將...
                               ...                        
17421    防地緣政治風險 台積電：政府應建立完整供應鏈盧志遠並說，希望政策能多開放吸引國外人才移民台灣...
17422    防斷鏈 台積電：新封裝關鍵材料、設備應產學合作加強創新研發〔記者洪友芳／台北報導〕台積電系統...
17423    防斷鏈 台積電：新封裝關鍵材料、設備應產學合作加強創新研發SEMICON Taiwan 20...
17424    盧志遠：台灣教授太低薪 每年應加薪15％〔記者洪友芳／台北報導〕SEMICON Taiwan...
17425    盧志遠：台灣教授太低薪 每年應加薪15％他認為，香港大學教授薪資比台灣高2倍、新加坡大學教授...
Length: 17426, dtype: object
       Related
0            0
1            0
2            0
3            0
4            0
...        ...
17421        1
17422        1
17423        1
17424        1
17425        0

[17426 rows x 1 columns]


In [ ]:
# 函數: 計算 Accuracy
def ACCscore_m_class(y_true, pred):
    accuracy_l = [ans.all() for ans in (pred == y_true)]
    accuracy = np.array(accuracy_l).mean()
    return accuracy


# 函數: 計算 Precision, Recall
def PRscore_m_class(y_true, pred):
    precision = precision_score(y_true, pred, zero_division=0)
    recall = recall_score(y_true, pred, zero_division=0)
    return precision, recall


# 函數: 計算 F1-score
def f1(precision, recall):
    f1_score = 0
    if (precision + recall) !=0:
        f1_score = (2 * precision * recall) / (precision + recall)
    return f1_score


# 函數: 一次同時計算所有評估指標
def score(y_true, pred):
    accuracy = ACCscore_m_class(y_true, pred)
    precision, recall = PRscore_m_class(y_true, pred)
    f1_score = f1(precision, recall)
    return accuracy, precision, recall, f1_score

### Modeling

### BERT Embedding

In [ ]:
def get_embedding_tensor(context_ary):

    embedding_tensor = torch.ones((context_ary.shape[0], 768))


    for i in range(context_ary.shape[0]):

        context = context_ary[i]
        bert_input = tokenizer(context, padding='max_length', max_length=512,
                               truncation=True, return_tensors="pt")


        attention_mask = bert_input['attention_mask'].to(device)
        input_id = bert_input['input_ids'].squeeze(1).to(device)
        final_inputs = {'input_ids': input_id, 'attention_mask': attention_mask}
        outputs = bert(**final_inputs)

        ## Only the embedding vector of [CLS] is taken
        pooler_output = outputs.last_hidden_state[0][0].reshape(768)
        embedding_tensor[i] = pooler_output.detach().cpu()

        gc.collect()
        torch.cuda.empty_cache()

    return embedding_tensor

In [ ]:
# 計算每個樣本各自的 BERT Embedding
context_ary = news_articles.values
embedding_tensor = get_embedding_tensor(context_ary)

# 儲存 BERT Embedding Tensor
torch.save(embedding_tensor, "/content/drive/MyDrive/IM 專題 - ESG rating system/embedding_tensor_seg_all.pt")

In [ ]:
# 載入 BERT Embedding Tensor
embedding_tensor = torch.load("/content/drive/MyDrive/IM 專題 - ESG rating system/embedding_tensor_seg_all.pt")
print(embedding_tensor.shape)

torch.Size([17426, 768])


### Calculate Class Weight

In [ ]:
def get_multiClass_weight(df, class_count=2):

    class_weight = []
#     class_count = df.value_counts().shape[0]
    for i in range(class_count):
        if i not in df.value_counts().index:
            class_weight.append(0)
        else:
            class_weight.append(df.shape[0]/(class_count*df.value_counts()[i]))

    return torch.tensor(class_weight, dtype=torch.float)

### Print out Class Weight

In [ ]:
def print_class_weight(task_label):
    print("<<各類別資料數>>")
    print(task_label.value_counts().sort_index())
    class_weight = get_multiClass_weight(task_label)
    print("<<目標變數與權重對應>>")
    print(f"目標變數共有 {len(class_weight)} 類")
    for c, w in zip(range(class_weight.shape[0]), class_weight):
        print(f"{c}: {round(float(w), 2)}")

In [ ]:
print_class_weight(label_data)

<<各類別資料數>>
Related
0          12497
1           4929
dtype: int64
<<目標變數與權重對應>>
目標變數共有 2 類
0: 0.7
1: 1.77


### Build Class: Dataset

In [ ]:
class Dataset(Dataset):
    def __init__(self, embeddings, label_list):
        self.embeddings = embeddings
        self.labels = label_list.to_numpy().reshape(-1)

    def __getLabels__(self):
        return (self.labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

### Model Architecture
- 實現一個神經網絡模型。該模型包含了兩層隱藏層架構並使用 dropout 機制

In [ ]:
class Classifier(nn.Module):
    def __init__(self, output_size=2, dropout_rate=0.5):
        super(Classifier, self).__init__()
        self.dropout = nn.Dropout(dropout_rate)
        self.linear_1 = nn.Linear(768, 256)
        #self.linear_11 = nn.Linear(512, 256)
        self.relu = nn.ReLU()
        self.linear_2 = nn.Linear(256, 64)
        self.linear_3 = nn.Linear(64, output_size)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, embeddings):
        output = self.linear_1(embeddings)
        output = self.relu(output)
        output = self.dropout(output)
        #output = self.linear_11(output)
        #output = self.relu(output)
        output = self.linear_2(output)
        output = self.relu(output)
        output = self.dropout(output)
        output = self.linear_3(output)
        output = self.softmax(output)

        return output

### Function for model training

In [ ]:
def train(model, class_weight, train_set, val_set, patience=10):
    train_loader = DataLoader(train_set, batch_size=config["batch_size"], shuffle=True)
    val_loader = DataLoader(val_set, batch_size=config["batch_size"], shuffle=False)
    # Loss Function
    criterion = nn.CrossEntropyLoss(weight=class_weight)

    # Optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=config["learning_rate"])

    # Putting the model on the GPU to run
    model = model.to(device)
    criterion = criterion.to(device)

    val_best_f1 = float(-np.inf)
    iteration_num = 0
    epoch_num = 0

    epochs = config["epochs"]

    while True:
        if iteration_num >= patience:
            break

        # Train and save training results
        loss_list = []
        accuracy_list = []
        precision_list = []
        recall_list = []
        f1_list = []

        # Set the model to the "training" state (Layers such as Dropout will be triggered only)
        model.train()

        for train_embeddings, train_labels in tqdm(train_loader):

            optimizer.zero_grad()

            train_embeddings = train_embeddings.to(device)
            train_labels = train_labels.to(device)
            #print("train_labels")
            #print(train_labels)

            output = model(train_embeddings)
            #print("output")
            #print(output)

            batch_loss = criterion(output, train_labels)

            # Back propagation
            batch_loss.backward()
            optimizer.step()

            output = output.cpu().detach()
            pred = output.argmax(dim=1).cpu()
            y_true = train_labels.cpu()

            #print("pred")
            #print(pred)
            #print("y_true")
            #print(y_true)

            # Calculate the performance of each batch and save it
            loss = batch_loss.item()
            accuracy, precision, recall, f1_score= score(y_true, pred)

            loss_list.append(loss)
            accuracy_list.append(accuracy)
            precision_list.append(precision)
            recall_list.append(recall)
            f1_list.append(f1_score)

        # Calculate the performance of each epoch and save it
        train_loss = np.array(loss_list).mean()
        train_accuracy = np.array(accuracy_list).mean()
        train_precision = np.array(precision_list).mean()
        train_recall = np.array(recall_list).mean()
        train_f1 = np.array(f1_list).mean()


        # Evaluate the current model performance using validate dataset
        loss_list = []
        accuracy_list = []
        precision_list = []
        recall_list = []
        pred_list = []
        f1_list = []

        with torch.no_grad():
            model.eval()
            for val_embeddings, val_labels in tqdm(val_loader):

                val_embeddings = val_embeddings.to(device)
                val_labels = val_labels.to(device)

                output = model(val_embeddings)

                batch_loss = criterion(output, val_labels)

                output = output.cpu()
                pred = output.argmax(dim=1).cpu()
                # sum_pred = torch.sum(pred, dim=1)
                # non_label_case_idx = (sum_pred < 1).nonzero()

                # if non_label_case_idx.shape[0] != 0:
                #     non_label_case = output[non_label_case_idx]
                #     max_col = torch.argmax(non_label_case, dim=-1)
                #     raw = non_label_case_idx.reshape(-1,1)
                #     pred_np = pred.numpy()
                #     for idx in range(raw.shape[0]):
                #         pred_np[raw[idx]][max_col[idx]] = True
                #     pred = torch.from_numpy(pred_np)
                # pred_list.extend(pred.numpy().astype('float32'))

                pred_list.extend(pred.tolist())
                y_true = val_labels.cpu()

                # Calculate the performance of each batch and save it
                loss = batch_loss.item()
                accuracy, precision, recall, f1_score = score(y_true, pred)

                loss_list.append(loss)
                accuracy_list.append(accuracy)
                precision_list.append(precision)
                recall_list.append(recall)
                f1_list.append(f1_score)

            # Calculate the performance of each epoch and save it
            val_loss = np.array(loss_list).mean()
            val_accuracy = np.array(accuracy_list).mean()
            val_precision = np.array(precision_list).mean()
            val_recall = np.array(recall_list).mean()
            val_f1 = np.array(f1_list).mean()


        if val_f1 > val_best_f1:
            ## val result
            val_best_f1 = val_f1
            val_best_loss = val_loss
            val_best_accuracy = val_accuracy
            val_best_precision = val_precision
            val_best_recall = val_recall
            ## train result
            train_best_f1 = train_f1
            train_best_loss = train_loss
            train_best_accuracy = train_accuracy
            train_best_precision = train_precision
            train_best_recall = train_recall
            ## other result
            best_model = model
            iteration_num = 0
            best_pred_list = pred_list
        else:
            iteration_num += 1

        epoch_num += 1
        # Print out the results of each Epoch
        print(f'Epochs: {epoch_num + 1} || Train Loss: {train_loss: .3f}, Accuracy: {train_accuracy: .3f}, Precision: {train_precision: .3f}, Recall:{train_recall: .3f}, F1: {train_f1: .3f} || Val Loss: {val_loss: .3f}, Accuracy: {val_accuracy: .3f}, Precision: {val_precision: .3f}, Recall:{val_recall: .3f}, Val F1: {val_f1: .3f} || Best F1: {val_best_f1: .3f}, Iter: {iteration_num: .0f}')

    train_result = [train_best_loss, train_best_accuracy, train_best_precision,
                    train_best_recall, train_best_f1]
    val_result = [val_best_loss, val_best_accuracy, val_best_precision,
                  val_best_recall, val_best_f1]

    return model, train_result, val_result, best_pred_list

### 10-Fold Cross-Validation

In [ ]:
def kfold_train_m_label(embedding_tensor, task_label, output_size, nfold=2):

    # 10 fold
    skf = StratifiedKFold(n_splits=nfold, shuffle=True, random_state=config["seed"])
    kfold_result = pd.DataFrame(columns=['Fold', 'train_loss', 'train_accuracy', 'train_precision',
                                    'train_recall', 'train_f1', 'val_loss', 'val_accuracy',
                                    'val_precision', 'val_recall', 'val_f1'])

    counts_true = pd.DataFrame(columns = ["num_label"])
    counts_pred = pd.DataFrame(columns = ["num_label"])

    pred_col_name = ["idx"]
    for o_idx in range(output_size):
        pred_col_name.append(f"{o_idx}")
    pred_result = pd.DataFrame(columns=pred_col_name)

    tp = 0
    tn = 0
    fp = 0
    fn = 0

    for fold_i, (train_fold, test_fold) in enumerate(skf.split(embedding_tensor, task_label.to_numpy().argmax(1))):

        print(f"Fold: {fold_i} | Train shape: {len(train_fold)} | Val shape: {len(test_fold)}")

        ## Preparation of training and validation datasets
        embedding_train, embedding_val = embedding_tensor[train_fold], embedding_tensor[test_fold]
        label_train, label_val = task_label.iloc[train_fold,:], task_label.iloc[test_fold,:]
        #print("label val =")
        #print(label_val)
        class_weight = print_class_weight(label_train)

        ## Actual number of labels
        actual_label_count = pd.DataFrame(sorted(Counter(label_val.sum(axis=1)).items()),
                                        columns = ["num_label", f"true_{fold_i}"])
        counts_true = pd.merge(counts_true, actual_label_count, how="outer", on = "num_label")

        ## Convert to Dataset data type
        train_set = Dataset(embedding_train, label_train)
        val_set = Dataset(embedding_val, label_val)

        print(f"Train 各類別資料量:", np.sum(train_set.__getLabels__(), axis=0))
        print(f"Val 各類別資料量:", np.sum(val_set.__getLabels__(), axis=0))
        model = Classifier(output_size=output_size)

        ## Model Training
        model, train_result, val_result, pred_list = train(model, class_weight, train_set, val_set)
        print("label val =")
        print(label_val)
        ## Record model evaluation results
        kfold_result = kfold_result.append([{"Fold": fold_i,
                                        "train_loss":train_result[0],
                                        "train_accuracy":train_result[1],
                                        "train_precision":train_result[2],
                                        "train_recall":train_result[3],
                                        "train_f1":train_result[4],
                                        "val_loss": val_result[0],
                                        "val_accuracy": val_result[1],
                                        "val_precision": val_result[2],
                                        "val_recall": val_result[3],
                                        "val_f1": val_result[4]}])


        print(len(label_val))
        pred_list = np.array(pred_list)
        print(len(pred_list))
        print(pred_list[1])
        """
        print(len(label_val))
        print(label_val[1])
        print()
        pred_list = np.array(pred_list)
        print(len(pred_list))
        print(pred_list[1])
        for data_index in range(len(label_val)):
          if label_val[data_index] == pred_list[data_index]:
            if label_val[data_index] == 1:
              tp += 1
            else:
              tn += 1
          else:
            if label_val[data_index] == 1:
              fp += 1
            else:
              fn += 1
        """

        ## Prediction results
        pred_list = np.array(pred_list)
        """
        new_record = {}
        new_record['idx'] = test_fold

        for o_idx in range(1, len(pred_col_name)):
            new_record[pred_col_name[o_idx]] = pred_list[:,o_idx-1]

        pred_result = pred_result.append(pd.DataFrame(new_record), ignore_index=True)
        print("-"*100)

        ## Predicted number of labels
        pred_label_count = pd.DataFrame(sorted(Counter(pred_list.sum(axis=1)).items()),
                          columns=["num_label", f"pred_{fold_i}"])
        counts_pred = pd.merge(counts_pred, pred_label_count, how="outer", on = "num_label")
        """

    #confusion_m = [tp, tn, fp, fn]

    return kfold_result, pred_result, counts_true, counts_pred

### Mainfunction

In [ ]:
kfold_result_DL, pred_result_DL, counts_true_DL, counts_pred_DL = kfold_train_m_label(embedding_tensor, label_data, 2)

Fold: 0 | Train shape: 8713 | Val shape: 8713
<<各類別資料數>>
Related
0          6289
1          2424
dtype: int64
<<目標變數與權重對應>>
目標變數共有 2 類
0: 0.69
1: 1.8
Train 各類別資料量: 2424
Val 各類別資料量: 2505


100%|██████████| 69/69 [00:00<00:00, 134.95it/s]


Epochs: 2 || Train Loss:  0.649, Accuracy:  0.711, Precision:  0.145, Recall: 0.012, F1:  0.021 || Val Loss:  0.627, Accuracy:  0.710, Precision:  0.000, Recall: 0.000, Val F1:  0.000 || Best F1:  0.000, Iter:  0


100%|██████████| 69/69 [00:00<00:00, 123.13it/s]


Epochs: 3 || Train Loss:  0.613, Accuracy:  0.722, Precision:  0.000, Recall: 0.000, F1:  0.000 || Val Loss:  0.606, Accuracy:  0.710, Precision:  0.000, Recall: 0.000, Val F1:  0.000 || Best F1:  0.000, Iter:  1


...

Epochs: 55 || Train Loss:  0.512, Accuracy:  0.793, Precision:  0.715, Recall: 0.470, F1:  0.560 || Val Loss:  0.517, Accuracy:  0.785, Precision:  0.564, Recall: 0.366, Val F1:  0.392 || Best F1:  0.405, Iter:  8


100%|██████████| 69/69 [00:00<00:00, 116.41it/s]


Epochs: 56 || Train Loss:  0.511, Accuracy:  0.793, Precision:  0.739, Recall: 0.447, F1:  0.551 || Val Loss:  0.517, Accuracy:  0.785, Precision:  0.565, Recall: 0.363, Val F1:  0.389 || Best F1:  0.405, Iter:  9


100%|██████████| 69/69 [00:00<00:00, 110.13it/s]
<ipython-input-95-111effbb12b4>:51: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  kfold_result = kfold_result.append([{"Fold": fold_i,


Epochs: 57 || Train Loss:  0.510, Accuracy:  0.796, Precision:  0.726, Recall: 0.470, F1:  0.566 || Val Loss:  0.516, Accuracy:  0.786, Precision:  0.557, Recall: 0.348, Val F1:  0.384 || Best F1:  0.405, Iter:  10
label val =
       Related
0            0
1            0
3            0
5            0
7            0
...        ...
17413        0
17417        0
17418        1
17420        1
17424        1

[8713 rows x 1 columns]
8713
8713
0


### Evaluation

In [ ]:
pred_result_DL = pred_result_DL.sort_values("idx")
pred_result_DL.reset_index(inplace=True, drop=True)
display(kfold_result_DL)
display(kfold_result_DL.mean()[2:])

,Fold,train_loss,train_accuracy,train_precision,train_recall,train_f1,val_loss,val_accuracy,val_precision,val_recall,val_f1
0,0,0.516102,0.789067,0.715218,0.421377,0.525655,0.507529,0.796762,0.602711,0.394236,0.448228
0,1,0.501230,0.808382,0.759199,0.473349,0.578854,0.500867,0.800188,0.579176,0.389796,0.447747
0,2,0.488438,0.824429,0.791413,0.517848,0.622070,0.488486,0.818949,0.640200,0.455709,0.516241
0,3,0.499525,0.807366,0.750075,0.477782,0.578530,0.497472,0.810367,0.625302,0.436075,0.504849
0,4,0.514757,0.789518,0.707140,0.431352,0.531201,0.537935,0.758914,0.645922,0.359404,0.439743
0,5,0.505863,0.800396,0.734242,0.457921,0.560267,0.523029,0.778502,0.645226,0.416472,0.469552
0,6,0.500361,0.808659,0.759682,0.475226,0.579963,0.496642,0.810239,0.584702,0.434490,0.474453
0,7,0.523171,0.781228,0.695236,0.404537,0.508070,0.515544,0.788762,0.567026,0.444811,0.464166
0,8,0.508218,0.798272,0.726185,0.468985,0.565816,0.502756,0.807091,0.570540,0.398169,0.443052
0,9,0.502308,0.804475,0.743980,0.471061,0.572381,0.512804,0.792024,0.602603,0.451040,0.483723


train_accuracy     0.801179
train_precision    0.738237
train_recall       0.459944
train_f1           0.562280
val_loss           0.508306
val_accuracy       0.796180
val_precision      0.606341
val_recall         0.418020
val_f1             0.469175
dtype: float64

In [ ]:
display(confusion_m_DL)

NameError: ignored

In [ ]:
from sklearn.metrics import confusion_matrix
y_true = label_data
y_pred = pred_result_DL
confusion_matrix(y_true, y_pred)

ValueError: ignored

In [ ]:
pred_result_DL

,idx,0,1


In [ ]:
label_data = pd.read_csv('/content/drive/MyDrive/IM 專題 - ESG rating system/label_data_seg_all.csv')

In [ ]:
label_data

,Related
0,0
1,0
2,0
3,0
4,0
...,...
17421,1
17422,1
17423,1
17424,1


In [ ]:
raw_data

,Index,Name,Content,Note,Company,Tag01,Tag02,Tag03,Tag04,URL,Unnamed: 10,Unnamed: 9,Keyword,xx,Related
0,NaN,「下半年實在看不清楚！」聯詠股東會烏雲裡仍有好消息：OLED Q4產能仍不足,驅動IC大廠聯詠（3034）今（8）日股東會，會中拍板去年現金股利51.5元，依照最新收盤價...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,NaN,「下半年實在看不清楚！」聯詠股東會烏雲裡仍有好消息：OLED Q4產能仍不足,股東會後，聯詠副董事長王守仁坦言，受到通膨因素、升息循環、俄烏衝突、中國封控等不確定因素干擾...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,NaN,「下半年實在看不清楚！」聯詠股東會烏雲裡仍有好消息：OLED Q4產能仍不足,面板從去年第三季就出現調節，聯詠第二季財測更正式預估毛利率將首度跌破50%，讓驅動IC市場傳...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,NaN,「下半年實在看不清楚！」聯詠股東會烏雲裡仍有好消息：OLED Q4產能仍不足,聯詠在第一季度法說即釋出能見度不明朗訊息，今日維持此看法，王守仁更近一步點出第三季不明朗的擔...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,NaN,「下半年實在看不清楚！」聯詠股東會烏雲裡仍有好消息：OLED Q4產能仍不足,王守仁列舉近期將因應需求不明朗而調整稼動率的大廠，例如三星將在六月關閉LCD廠、轉做OLED...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17421,198,防地緣政治風險 台積電：政府應建立完整供應鏈,盧志遠並說，希望政策能多開放吸引國外人才移民台灣，像AI晶片大廠輝達員工中有四成印度人；目前...,NaN,NaN,多樣性與共榮 - 1,勞工關係 - 1,NaN,NaN,NaN,NaN,NaN,旺宏,NaN,1
17422,199,防斷鏈 台積電：新封裝關鍵材料、設備應產學合作加強創新研發,〔記者洪友芳／台北報導〕台積電系統整合前瞻研發副總經理余振華今指出，地緣政治使得生產製造走向...,NaN,NaN,供應鏈管理 - 1,NaN,NaN,NaN,NaN,NaN,NaN,旺宏,NaN,1
17423,199,防斷鏈 台積電：新封裝關鍵材料、設備應產學合作加強創新研發,SEMICON Taiwan 2023國際半導體展今天下午舉行「半導體研發大師論壇」，由國科...,NaN,NaN,多樣性與共榮 - 1,NaN,NaN,NaN,NaN,NaN,NaN,旺宏,NaN,1
17424,200,盧志遠：台灣教授太低薪 每年應加薪15％,〔記者洪友芳／台北報導〕SEMICON Taiwan 2023國際半導體展今天下午舉行「半導...,NaN,NaN,薪酬與福利 - 1,NaN,NaN,NaN,NaN,NaN,NaN,旺宏,NaN,1


In [ ]:
count = 0
c = 0
for i in range(len(label_data)):
  if label_data["Related"][i] == 1:
    if type(raw_data["Company"][i]) != str:
      count += 1
    else:
      c += 1

print("有關但無公司標籤的段落數=",count)
print("有關且有公司標籤的段落數=",c)
print("有關但無公司標籤的比例=", count/(count+c))

有關但無公司標籤的段落數= 1550
有關且有公司標籤的段落數= 3379
有關但無公司標籤的比例= 0.31446540880503143
